In [1]:
from os import listdir
import cv2
import numpy as np
import pandas as pd
import os
import sys
module_path ='/home/Mingke.Chen/OwO/hh'
sys.path.append(module_path)
import copy
from tensorflow.contrib import slim
import tensorflow as tf
#from nets import inception_v4
from nets import alexnet
from tensorflow.python.ops import init_ops
tf.logging.set_verbosity('ERROR')
from nets.batch_data_generater_hson import batch_data_generater
from nets.batch_data_generater_hson_valid import batch_data_generater as batch_data_generater_valid
from keras.preprocessing.image import ImageDataGenerator


class VeriSee_training():
    def __init__(self,logdir,model_name):
        self.init_checking_point_and_log_dir(logdir,model_name)

    
        X_data=tf.placeholder(tf.float32, (None, 64, 64, 3))
        y_label=tf.placeholder(tf.float32, (None,3))
        self.X_data=X_data
        self.y_label=y_label

        
        self.init_training_parameter()
        
        self.predict_tensor,self.sess=self.your_model(self.X_data)

        
        self.init_summary_op()
        self.sess.run(tf.global_variables_initializer())

    def init_summary_op(self):
        self.writer_master = tf.summary.FileWriter(self.log_path, self.sess.graph)
        self.init_summary=False
 
        self.value_feeder=tf.placeholder(tf.float32)
        
        
    def init_training_parameter(self):
        
        self.global_step_net= tf.Variable(0, name='global_step_net', trainable=False)
        self.dropout_keep_prob_tensor=tf.placeholder(tf.float32)
        self.dropout_keep_prob=1.0
        self.learning_rate=0.001
        self.learning_rate_tensor=tf.placeholder(tf.float32)
      
        self.global_step=0
        self.checkpoints=0

        
    def init_checking_point_and_log_dir(self,logdir,model_name):
        checking_point_and_log=os.path.join(logdir,model_name)
        if not os.path.exists(checking_point_and_log):
            os.mkdir(checking_point_and_log)
            
        self.checking_point_path=os.path.join(checking_point_and_log,'model')
        if not os.path.exists(self.checking_point_path):
            os.mkdir(self.checking_point_path)
        self.checking_point_path=os.path.join(self.checking_point_path,model_name)
        
        self.log_path=os.path.join(checking_point_and_log,'log')
        if not os.path.exists(self.log_path):
            os.mkdir(self.log_path)
        
        self.text_report_path=os.path.join(checking_point_and_log,model_name+'.csv')
        
        
    def your_model(self,X_data):

        
        net,endpoint=alexnet.alexnet_v2(X_data,
               num_classes=3,
               is_training=True,
               dropout_keep_prob=0.7,
               spatial_squeeze=False,
               scope='alexnet_v2')
        
        
        
        
        self.train_loss=slim.losses.softmax_cross_entropy(net,self.y_label)
        
        optimizer= tf.train.GradientDescentOptimizer(self.learning_rate_tensor)
        
        self.train_op=slim.learning.create_train_op(self.train_loss, optimizer,self.global_step_net)
        
        
        
        #gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.4,allow_growth = True)
        gpu_options = tf.GPUOptions(allow_growth = True)
        sess=tf.Session(config=tf.ConfigProto(log_device_placement=True,gpu_options=gpu_options))
        #sess=tf.Session()

        return net,sess 

        
        #return self.train_op
    def predict(self,data,label,max_batch=100,accuracy=False):
        test_data_length=len(data)
        split_batchs=int((test_data_length-0.1)/max_batch)
        for start_partition in range(split_batchs+1):
            start_index=start_partition*max_batch
            prediction_batch=self.sess.run(self.predict_tensor,feed_dict=
                                  {self.X_data:data[start_index:(start_index+max_batch)],
                                  self.y_label:label[start_index:(start_index+max_batch)],
                                  self.dropout_keep_prob_tensor:self.dropout_keep_prob,
                                  self.learning_rate_tensor:self.learning_rate
                                  })
            
            prediction_batch=np.asarray(prediction_batch)
            if start_partition == 0 :
                prediction=prediction_batch
            else:
                prediction=np.concatenate([prediction,prediction_batch])
        
        
        if accuracy==True:
            accuracy=np.mean(np.equal(prediction.argmax(1),label.argmax(1)))
            with tf.Session() as sess:
                cross_entropy_loss=slim.losses.softmax_cross_entropy(tf.convert_to_tensor(prediction)
                                                                     ,tf.convert_to_tensor(label))
                cross_entropy_loss_=sess.run(cross_entropy_loss)

            return accuracy,cross_entropy_loss_
        else:
            return prediction.argmax(1)
    
    def predict_yours(self,data,max_batch=100):
        test_data_length=len(data)
        split_batchs=int((test_data_length-0.1)/max_batch)
        for start_partition in range(split_batchs+1):
            start_index=start_partition*max_batch
            prediction_batch=self.sess.run(self.predict_tensor,feed_dict=
                                  {self.X_data:data[start_index:(start_index+max_batch)],
                                  self.dropout_keep_prob_tensor:self.dropout_keep_prob,
                                  self.learning_rate_tensor:self.learning_rate
                                  })
            
            prediction_batch=np.asarray(prediction_batch)
            if start_partition == 0 :
                prediction=prediction_batch
            else:
                prediction=np.concatenate([prediction,prediction_batch])

        return prediction.argmax(1)
      
   
    
    def save_checking_point(self):
        Saver=tf.train.Saver()
        Saver.save(self.sess, self.checking_point_path, global_step=self.checkpoints)
        
    def restor_checkin_gpoint(self,checking_point_path):
        Saver=tf.train.Saver()
        Saver.restore(self.sess, checking_point_path)
    

        

    
  
        
    
    def write_summary_and_report(self,test_data,test_label):
    
        loss=self.train_losses_

        accuracy,validation_losses_subnets=self.predict(test_data,test_label,accuracy=True)

        
        report_data=[accuracy,loss,validation_losses_subnets]
   
 
        
       
        if self.init_summary == False:
            self.summary_manager=[]
            with tf.variable_scope('Report_h') as scope:
                
                self.summary_manager.append(tf.summary.scalar('Accuracy', self.value_feeder))
                self.summary_manager.append(tf.summary.scalar('Loss', self.value_feeder))
                self.summary_manager.append(tf.summary.scalar('Validation_losses', self.value_feeder))

                       
            self.init_summary=True
        
        for summary_index in range(len(report_data)):
          
            summary=self.sess.run(self.summary_manager[summary_index],
                                  feed_dict={self.value_feeder:report_data[summary_index]})
            self.writer_master.add_summary(summary,self.checkpoints)
   
        self.writer_master.flush()
   
        
        
    
    def validation(self,test_data,test_label,save=True,init=False):

        if save== True:
            self.save_checking_point()
        self.write_summary_and_report(test_data,test_label)
        self.checkpoints+=1
        
        return 0

        
    
    
    
    def train(self,X_data,y_label):

        
        _,self.train_losses_=self.sess.run([
                                            self.train_op,self.train_loss],
                                                       feed_dict={ 
                                                       self.X_data:X_data,self.y_label:y_label,
                                                      self.dropout_keep_prob_tensor:self.dropout_keep_prob,
                                                       self.learning_rate_tensor:self.learning_rate})
        
        

        
       
        self.global_step+=1
        return 1
     

    def fit(self,train_batch_generater,test_data,test_label,steps,valid_data_per_step,times=1,learning_rate_decay=False,
           learning_rate=False,dropout_keep_prob=False):
        if learning_rate != False:
            if type(learning_rate) is float:
                self.learning_rate=learning_rate
            else:
                raise ValueError("learning_rate must be float")
                
        if dropout_keep_prob != False:
            if type(dropout_keep_prob) is float:
                if dropout_keep_prob <= 1.0:
                    self.dropout_keep_prob=dropout_keep_prob
                else:
                    raise ValueError("dropout_keep_prob must smaller than or equal to 1.0")
            else:
                raise ValueError("dropout_keep_prob must be float")
 
                
        
        for times_ in list(range(times)):
            for step in list(range(steps)):
                X_data,y_label=next(train_batch_generater)
                #print (self.global_step)
                X_data=X_data
                #prediction = self.train(X_data,y_label)
                self.train(X_data,y_label)
                if type(learning_rate_decay) is int:
                    if self.global_step % learning_rate_decay == 0 :
                        if self.global_step != 0 :
                            self.learning_rate=self.learning_rate/2
                            
                if self.global_step % valid_data_per_step == 0 :
                    test_data=test_data
                    print (self.global_step)
                    _=self.validation(test_data,test_label)
                    
                    
                    ##255
                    
        
    
    


Using TensorFlow backend.


In [2]:
logdir='/home/Mingke.Chen/DR/log/train'
model_name='hson155'
filepath='/raidHDD/experimentData/money/upup10'
testfile='/raidHDD/experimentData/money/orign/07202215_re'

In [4]:
batch_generater=batch_data_generater()
train_batch=batch_generater.get_batch(filepath,batch_size=10,batch_type='balance')
test_X,test_y=next(train_batch)

In [5]:
test_file=batch_data_generater_valid()
test_batch=test_file.get_batch(testfile,batch_size=310,batch_type='balance')
test_X,test_y=next(test_batch)

In [6]:
train_op=VeriSee_training(logdir,model_name)

In [1]:
train_op.fit(train_batch,test_X,test_y,steps=100000,valid_data_per_step=4000,
            times=40,dropout_keep_prob=0.5,learning_rate=0.001)